In [3]:
import pandas as pd

In [6]:
pip install kaggle

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d notshrirang/spotify-million-song-dataset

spotify-million-song-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
from zipfile import ZipFile
dataset='/content/spotify-million-song-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("Dataset is extracted")

Dataset is extracted


In [70]:
df=pd.read_csv('/content/spotify_millsongdata.csv')

In [71]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [72]:
df.shape

(57650, 4)

In [73]:
#Checking for any null values
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [74]:
#Removing the link column

df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [75]:
# df=data.sample(5000)
# df.shape
# df['text'][0]
df.head()

,artist,song,text
0,Enya,I Will Find You,Hope is your survival \r\nA captive path I le...
1,Bosson,Fallin In Love,My doctor says I need no cure this \r\nTime T...
2,Rod Stewart,Blind Prayer,I was born and raised the only son of a lawyer...
3,Jim Croce,I Am Who I Am,Perhaps I'll never show this world all I could...
4,Jim Croce,"Another Day, Another Town",My dusty feet have kicked the stones \r\nBetw...


Text Cleaning

In [53]:
df['text']=df['text'].str.lower().replace(r'^\W\S',' ').replace(r'\n',' ',regex=True)  #Removing all tab space characters

In [54]:
df.head()

,artist,song,text
0,Frank Sinatra,All The Way,when somebody loves you \r it's no good unles...
1,Green Day,Blitzkrieg Bop,hey ho! lets go! \r hey ho! lets go! \r hey ...
2,Natalie Grant,Waiting For A Prayer,he's given hope through the ages \r give sigh...
3,Dolly Parton,I Can't Help Myself,"oh sugar pie, honey bunch \r you know that i ..."
4,Weird Al Yankovic,Polka Party!,you could have a big dipper \r going up and d...


In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

In [13]:
stemmer=PorterStemmer()

In [14]:
def token(text):
  token=nltk.word_tokenize(text)
  a=[stemmer.stem(w) for w in token]
  return " ".join(a)

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
token("What You See Is What You Don't Get")

"what you see is what you do n't get"

In [35]:
df['text']=df['text'].apply(lambda x: token(x))

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [55]:
matrix=tfid.fit_transform(df['text'])

In [56]:
similar=cosine_similarity(matrix)
similar

array([[1.00000000e+00, 3.06056074e-03, 1.82022829e-02, ...,
        2.19373139e-02, 3.61284828e-03, 6.36507070e-03],
       [3.06056074e-03, 1.00000000e+00, 2.58078916e-02, ...,
        0.00000000e+00, 0.00000000e+00, 3.77945584e-04],
       [1.82022829e-02, 2.58078916e-02, 1.00000000e+00, ...,
        8.51030513e-03, 1.63123823e-03, 0.00000000e+00],
       ...,
       [2.19373139e-02, 0.00000000e+00, 8.51030513e-03, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.61284828e-03, 0.00000000e+00, 1.63123823e-03, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.36507070e-03, 3.77945584e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

Song Recommender Function

In [57]:
def recommender(song_name):
  idx=df[df['song']==song_name].index[0]
  distance=sorted(list(enumerate(similar[idx])),reverse=True, key= lambda x:x[1])
  song=[]
  for s_id in distance[1:5]:
    song.append(df.iloc[s_id[0]].song)
  return song


In [77]:
recommender('Flight 602')

['Talking Book', 'I Woke Up', 'Dance The Night Away', 'Looking At The Rain']